In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join("..")))
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from src.data_loader import load_sales, load_purchases, load_products
from src.demand_forecast import calculate_forecast
from src.stock_manager import calculate_min_stock
from src.purchase_suggester import suggest_purchases
from src.supplier_selector import select_suppliers


In [ ]:
# === Opciones disponibles ===
coverage_days_options = [7, 15, 21, 30]
forecast_methods = ["mean", "rolling", "weighted", "trend"]

# === Widgets ===
coverage_dropdown = widgets.Dropdown(
    options=coverage_days_options,
    value=7,
    description="Cobertura (días):",
    style={'description_width': 'initial'}
)

method_dropdown = widgets.Dropdown(
    options=forecast_methods,
    value="mean",
    description="Método forecast:",
    style={'description_width': 'initial'}
)

run_button = widgets.Button(
    description="Generar Sugerido",
    button_style="success"
)

output = widgets.Output()

# === Función a ejecutar ===
def run_forecast(b):
    with output:
        output.clear_output()
        try:
            coverage_days = coverage_dropdown.value
            method = method_dropdown.value

            print(f"⚡ Ejecutando forecast con {coverage_days} días y método '{method}'...\n")

            # === Cargar datos ===
            sales = load_sales("../data/sales.csv")
            stock = load_products("../data/products.csv")
            purchases = load_purchases("../data/compras.csv")  # ojo: revisa si es otro loader

            # === Forecast ===
            forecast = calculate_forecast(
                sales,
                days=60,      # últimos 60 días para suavizar outliers
                method=method,
                window=7
            )

            # === Sugerido ===
            suggestion = suggest_purchases(
                forecast,
                stock,
                coverage_days=coverage_days
            )

            # === Proveedores ===
            suppliers = select_suppliers(purchases)

            final = suggestion.merge(suppliers, on="product_id", how="left")

            if final.empty:
                print("⚠️ No hay datos para mostrar (revisa tu CSV o filtros).")
            else:
                display(final.head(20))

        except Exception as e:
            print("❌ Error en la ejecución:", e)

# Conectar botón a función
run_button.on_click(run_forecast)

# Mostrar widgets
display(coverage_dropdown, method_dropdown, run_button, output)